In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt



KeyboardInterrupt: 

In [ ]:
df = pd.read_excel(r"D:\Final_Project\LSTM\final_synthetic_lstm.xlsx")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['patient_id'].nunique()


In [ ]:
df.groupby('patient_id')['hour'].nunique().describe()


In [ ]:
df.isnull().sum()


In [ ]:
import seaborn as sns
vitals = [
    "heart_rate",
    "systolic_bp",
    "diastolic_bp",
    "spo2",
    "respiratory_rate",
    "temperature"
]

plt.figure(figsize=(15, 10))
for i, col in enumerate(vitals, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df[col], bins=40, kde=True)
    plt.title(col.replace("_", " ").title())

plt.tight_layout()
plt.show()


📌 Interpretation:

HR: right-skew → tachycardia cases

SpO₂: left tail → hypoxia

RR: wider spread → respiratory distress

In [ ]:
sample_patient = df['patient_id'].iloc[0]

patient_df = df[df['patient_id'] == sample_patient]

plt.figure(figsize=(12, 6))
plt.plot(patient_df['hour'], patient_df['heart_rate'], label="Heart Rate")
plt.plot(patient_df['hour'], patient_df['spo2'], label="SpO₂")
plt.xlabel("Hour")
plt.ylabel("Value")
plt.title("ICU Vital Trends – Sample Patient")
plt.legend()
plt.show()


“Vitals show temporal dependency, justifying recurrent neural networks.”

In [ ]:
import tensorflow as tf
print(tf.__version__)

# Correlation Analysis

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(
    df[vitals].corr(),
    annot=True,
    cmap="coolwarm",
    fmt=".2f"
)
plt.title("Vital Signs Correlation Matrix")
plt.show()


📌 Expected Findings:

HR ↑ ↔ RR ↑

SpO₂ ↓ ↔ HR ↑

BP moderately independent

In [ ]:
#Population-Level Temporal Patterns
mean_trends = df.groupby('hour')[vitals].mean()

mean_trends.plot(figsize=(12, 6))
plt.title("Average ICU Vital Trends Across Patients")
plt.xlabel("Hour")
plt.ylabel("Mean Value")
plt.grid()
plt.show()


“Aggregated trends reveal subtle deterioration patterns across time.”

In [ ]:
FEATURES = [
    "heart_rate",
    "systolic_bp",
    "diastolic_bp",
    "spo2",
    "respiratory_rate",
    "temperature"
]


In [ ]:
scaler = MinMaxScaler()
df[FEATURES] = scaler.fit_transform(df[FEATURES])

X, y = [], []

In [ ]:
for pid in df.patient_id.unique():
    patient_df = df[df.patient_id == pid].sort_values("hour")
    
    X.append(patient_df[FEATURES].iloc[:24].values)   # past 24 hrs
    y.append(patient_df["heart_rate"].iloc[24])       # next hour HR

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)  # (patients, 24, 6)
print("y shape:", y.shape)

# LSTM Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [ ]:
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(24, 6)),
    Dropout(0.3),
    LSTM(64),
    Dense(32, activation="relu"),
    Dense(1)   # Next-hour heart rate
])

model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

model.summary()


In [ ]:
model.fit(
    X, y,
    epochs=30,
    batch_size=64,
    validation_split=0.2
)


In [ ]:
import joblib
model.save("icu_lstm_hr_model.h5")
joblib.dump(scaler, "icu_scaler.joblib")


# ICU Deterioration Logic

In [ ]:
def icu_risk_level(pred_hr, spo2):
    """
    Simple clinical rule-based risk mapping
    """
    if pred_hr > 130 or spo2 < 90:
        return "🔴 Critical"
    elif pred_hr > 100 or spo2 < 94:
        return "🟠 Warning"
    else:
        return "🟢 Stable"


In [ ]:
sample_cases = [
    (85, 97),
    (105, 93),
    (130, 88)
]

for hr, spo2 in sample_cases:
    print(f"HR={hr}, SpO2={spo2} → Risk={icu_risk_level(hr, spo2)}")